<a href="https://colab.research.google.com/github/Edfred1/Contextual-Music-Crafter/blob/main/CMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation and Setup

This section covers cloning the repository and installing the necessary dependencies.

In [ ]:
# Clone the Git repository
!git clone https://github.com/Edfred1/Contextual-Music-Crafter.git
%cd Contextual-Music-Crafter

# Install dependencies
!pip install -r requirements.txt

### Configuration

**⚠️ IMPORTANT: You must run the code cell below to see the configuration editor!**

1. **Click on the code cell below** (the one that starts with `from ruamel.yaml import YAML`)
2. **Press the "Run" button** (or press Shift+Enter) to execute the cell
3. **The configuration editor text field will appear below** - that's where you edit your settings!

**⚠️ Use the configuration editor widget (text field), NOT the code itself!**

Here, we configure the `config.yaml` file. This file contains settings for the AI API, musical parameters, and instrument definitions. 

**After running the cell below, scroll down to find the configuration editor text field** - edit your settings there (including your Google AI API key). Do NOT edit the code in the cell - it's just for loading the default configuration!

In [ ]:
from ruamel.yaml import YAML
from pathlib import Path
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from io import StringIO

# Show instructions FIRST - before any code
print("=" * 80)
print("CONFIGURATION EDITOR")
print("=" * 80)
print("\n✅ This cell has been executed! The configuration editor should appear below.")
print("\n💡 IMPORTANT: Edit your configuration in the text field below (scroll down to see it).")
print("   Do NOT edit the code above - use the configuration editor widget instead!")
print("   All settings are visible and can be changed in relation to each other.")
print("   Use 'Validate' to check the syntax before saving.")
print("\n" + "=" * 80 + "\n")

# Path to config.yaml
config_path = Path('config.yaml')

# Load existing config (preserve comments); create a minimal starter if missing
yaml = YAML(typ='rt')
yaml.preserve_quotes = True
yaml.indent(mapping=2, sequence=4, offset=2)

doc = None
if config_path.exists():
    try:
        with config_path.open('r', encoding='utf-8') as f:
            doc = yaml.load(f)
    except Exception:
        doc = None

if doc is None:
    doc = yaml.load("""
api_key:
  - "YOUR_GOOGLE_AI_API_KEY_1"
model_name: "gemini-2.5-pro"
temperature: 0
lyrics_temperature: 0.0
enable_hotkeys: 1
stage2_invalid_retries: 5
pass_raw_prompt_to_stages: 0
inspiration: "A new track"
genre: "House"
bpm: 125
key_scale: "C minor"
automation_settings:
  use_pitch_bend: 0
  use_sustain_pedal: 0
  use_cc_automation: 0
  allowed_cc_numbers: [1, 10, 11, 74]
max_output_tokens: 65536
context_window_size: -1
use_call_and_response: 0
number_of_iterations: 1
time_signature:
  beats_per_bar: 4
  beat_value: 4
instruments:
  - name: "Drums"
    program_num: 10
    role: "drums"
  - name: "Bass"
    program_num: 39
    role: "bass"
""")

# Convert current config to YAML string for editing
stream = StringIO()
yaml.dump(doc, stream)
config_yaml_text = stream.getvalue()

# Create textarea with current config (larger for better overview)
config_textarea = widgets.Textarea(
    value=config_yaml_text,
    placeholder='Edit YAML configuration here...',
    description='',
    disabled=False,
    layout=widgets.Layout(width='100%', height='600px'),
    style={'font_family': 'monospace', 'font_size': '13px'}
)

# Status output widget
status_output = widgets.Output()

# Display the configuration editor FIRST (before buttons and help)
# This makes it clear that users should edit here, not in the code above
display(HTML("""
<div style="background-color: #fff3cd; border: 3px solid #ffc107; padding: 15px; margin: 20px 0; border-radius: 5px;">
<h2 style="color: #856404; margin-top: 0;">⚠️ EDIT YOUR CONFIGURATION HERE ⚠️</h2>
<p style="color: #856404; font-size: 16px; font-weight: bold;">
📝 Use the text field below to edit your settings (including API key).<br>
❌ Do NOT edit the code above - it's just for loading the default configuration!
</p>
</div>
"""))
display(config_textarea)

# Validate button callback
def validate_config(btn):
    with status_output:
        status_output.clear_output()
        try:
            # Parse the edited YAML
            edited_doc = yaml.load(config_textarea.value)
            
            print("=" * 80)
            print("✓ YAML syntax is valid!")
            print("=" * 80)
            
            # Validate required fields
            warnings = []
            if not edited_doc.get('api_key') or not any(edited_doc.get('api_key', [])):
                warnings.append("⚠️  API key is missing or empty!")
            elif any(key == "YOUR_GOOGLE_AI_API_KEY_1" or "YOUR_GOOGLE_AI_API_KEY" in str(key) for key in edited_doc.get('api_key', [])):
                warnings.append("⚠️  Please replace the placeholder API key!")
            
            if warnings:
                print("\n⚠️  WARNINGS:")
                for w in warnings:
                    print(f"   {w}")
            else:
                print("\n✓ All required fields are present.")
            
            print("\n📋 Current Settings:")
            print(f"   • Model: {edited_doc.get('model_name', 'N/A')}")
            print(f"   • Genre: {edited_doc.get('genre', 'N/A')}")
            print(f"   • BPM: {edited_doc.get('bpm', 'N/A')}")
            print(f"   • Key/Scale: {edited_doc.get('key_scale', 'N/A')}")
            print(f"   • Instruments: {len(edited_doc.get('instruments', []))}")
            print(f"   • Iterations: {edited_doc.get('number_of_iterations', 'N/A')}")
            print(f"   • Temperature: {edited_doc.get('temperature', 'N/A')}")
            print(f"   • Max Output Tokens: {edited_doc.get('max_output_tokens', 'N/A')}")
            
        except Exception as e:
            print("=" * 80)
            print("✗ YAML SYNTAX ERROR")
            print("=" * 80)
            print(f"\n❌ Error: {str(e)}")
            print("\n💡 Please check the YAML syntax:")
            print("   • Indentation must be correct (2 spaces per level)")
            print("   • Lists start with '-' and are indented")
            print("   • Strings in quotes if they contain special characters")
            print("   • Numbers without quotes")
            print("   • Don't forget colons after keys")

# Save button callback
def save_config(btn):
    with status_output:
        status_output.clear_output()
        try:
            # Parse the edited YAML
            edited_doc = yaml.load(config_textarea.value)
            
            # Validate required fields
            warnings = []
            if not edited_doc.get('api_key') or not any(edited_doc.get('api_key', [])):
                warnings.append("⚠️  WARNING: API key is missing or empty!")
            elif any(key == "YOUR_GOOGLE_AI_API_KEY_1" or "YOUR_GOOGLE_AI_API_KEY" in str(key) for key in edited_doc.get('api_key', [])):
                warnings.append("⚠️  WARNING: Please replace the placeholder API key!")
            
            # Save to file
            with config_path.open('w', encoding='utf-8') as f:
                yaml.dump(edited_doc, f)
            
            print("=" * 80)
            print("✓ Configuration saved successfully!")
            print("=" * 80)
            print(f"\n📁 Saved to: {config_path.absolute()}")
            
            if warnings:
                print("\n" + "\n".join(warnings))
            
            print("\n📋 Saved Settings:")
            print(f"   • Model: {edited_doc.get('model_name', 'N/A')}")
            print(f"   • Genre: {edited_doc.get('genre', 'N/A')}")
            print(f"   • BPM: {edited_doc.get('bpm', 'N/A')}")
            print(f"   • Key/Scale: {edited_doc.get('key_scale', 'N/A')}")
            print(f"   • Instruments: {len(edited_doc.get('instruments', []))}")
            print(f"   • Iterations: {edited_doc.get('number_of_iterations', 'N/A')}")
            
        except Exception as e:
            print("=" * 80)
            print("✗ ERROR saving configuration")
            print("=" * 80)
            print(f"\n❌ Error: {str(e)}")
            print("\n💡 Please check the YAML syntax:")
            print("   • Indentation must be correct (2 spaces per level)")
            print("   • Lists start with '-' and are indented")
            print("   • Strings in quotes if they contain special characters")
            print("   • Numbers without quotes")
            print("   • Don't forget colons after keys")
            print("\n💡 Tip: Use 'Validate' first to find errors!")

# Create buttons
validate_button = widgets.Button(
    description='🔍 Validate',
    button_style='info',
    layout=widgets.Layout(width='150px', height='40px', margin='5px')
)

save_button = widgets.Button(
    description='💾 Save Configuration',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px', margin='5px')
)

validate_button.on_click(validate_config)
save_button.on_click(save_config)

# Button container
button_box = widgets.HBox([validate_button, save_button])

# Display buttons and status (textarea already displayed above)
display(button_box)
display(status_output)

# Show help text
help_text = """
<details>
<summary><b>📖 Help: Important Settings</b></summary>

<h4>API Configuration:</h4>
<ul>
  <li><code>api_key</code>: Google AI API key (required, multiple keys possible as a list)</li>
  <li><code>model_name</code>: "gemini-2.5-pro" (best quality) or "gemini-2.5-flash" (faster/cheaper)</li>
  <li><code>temperature</code>: 0.0 = deterministic, 2.0 = very creative</li>
  <li><code>lyrics_temperature</code>: Creativity for lyrics generation</li>
</ul>

<h4>Musical Parameters:</h4>
<ul>
  <li><code>inspiration</code>: Description for the composition</li>
  <li><code>genre</code>: Music style (e.g., "House", "Techno", "Rock", "Jazz")</li>
  <li><code>bpm</code>: Tempo (e.g., 120-140, House=120-130, Techno=130-140)</li>
  <li><code>key_scale</code>: Musical key and scale (e.g., "C minor", "A major", "F# dorian")</li>
</ul>

<h4>Instruments:</h4>
<ul>
  <li>Each instrument needs: <code>name</code>, <code>program_num</code> (1-128), <code>role</code></li>
  <li>Roles: "drums", "bass", "pads", "lead", "melody", "chords", "vocal", "fx", etc.</li>
  <li>Example for multiple instruments:
<pre>instruments:
  - name: "Drums"
    program_num: 10
    role: "drums"
  - name: "Bass"
    program_num: 39
    role: "bass"
  - name: "Piano"
    program_num: 1
    role: "pads"</pre>
  </li>
</ul>

<h4>Additional Settings:</h4>
<ul>
  <li><code>number_of_iterations</code>: Number of songs to generate</li>
  <li><code>max_output_tokens</code>: Maximum tokens per response (higher = longer outputs, more cost)</li>
  <li><code>context_window_size</code>: -1 = dynamic, 0 = none, >0 = fixed number</li>
  <li><code>time_signature</code>: Time signature (beats_per_bar: 4, beat_value: 4 = 4/4 time)</li>
  <li><code>automation_settings</code>: MIDI automation (Pitch Bend, Sustain, CC)</li>
</ul>

<h4>💡 Tips:</h4>
<ul>
  <li>Use "Validate" first to find syntax errors</li>
  <li>Multiple API keys can be specified as a list (automatic rotation)</li>
  <li>All settings can be adjusted simultaneously in relation to each other</li>
</ul>
</details>
"""
display(HTML(help_text))

### Running a Python Program from the Repository

This cell lists the Python programs found in the cloned repository and prompts you to select one to execute. The selected program will then be run based on the configuration.

In [ ]:
import os

# Get a list of all files and directories in the cloned repository
files_and_directories = os.listdir('.')

# Filter for Python files
python_files = [f for f in files_and_directories if f.endswith('.py')]

if not python_files:
    print("No Python files found in the repository.")
else:
    print("Found Python files:")
    for i, filename in enumerate(python_files):
        print(f"{i+1}. {filename}")

    # Prompt the user to select a file
    while True:
        try:
            choice = int(input(f"Enter the number of the file to execute (1-{len(python_files)}): "))
            if 1 <= choice <= len(python_files):
                selected_file = python_files[choice - 1]
                print(f"Executing {selected_file}...")
                # Execute the selected Python file
                !python {selected_file}
                break
            else:
                print("Invalid input. Please enter a number from the list.")
        except ValueError:
            print("Invalid input. Please enter a number.")